## Dataloader

In [1]:
import numpy as np
from tqdm import tqdm
import torch
import os

In [2]:
import torch.utils.data as data

class PointCloudDataset(data.Dataset):

    def __init__(self,target_files,phase,transform):
        self.transform=transform
        self.target_files=target_files

    def __len__(self):
        return len(self.target_files)

    def __getitem__(self,index):
        pcd,one_hot = self.pull_item(index)
        return pcd,one_hot

    def pull_item(self,index):
        self.path=self.target_files[index]

        #label
        label = int(self.path[2])

        #load_pcd
        pcd = np.load('DATA/MSRAction_npz_pp/' + self.path,allow_pickle=True)['point_clouds']
        pcd = torch.from_numpy(pcd.astype(np.float32)).clone()
        #pcd=self.transform(pcd)
        return pcd, label

In [3]:
input_dir = 'DATA/MSRAction_npz_pp'
files = os.listdir(input_dir)

train_files = [a for a in files if a[10] == '3']
test_files = [a for a in files if a[10] != '3']

In [4]:
n_samples = len(train_files)
train_size = int(n_samples * 0.8)
val_size = n_samples - train_size

trainval_dataset = PointCloudDataset(target_files=train_files,phase='train',transform=None)
train_dataset, val_dataset = torch.utils.data.random_split(trainval_dataset, [train_size,val_size])
test_dataset = PointCloudDataset(target_files=test_files,phase='train',transform=None)

## MODEL

In [8]:
from torch_geometric.nn import global_max_pool
import torch.nn as nn

In [9]:
class InputTNet(nn.Module):
    def __init__(self):
        super(InputTNet, self).__init__()
        self.input_mlp = nn.Sequential(
            nn.Linear(3, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, 1024), nn.BatchNorm1d(1024), nn.ReLU(),
        )
        self.output_mlp = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(),
            nn.Linear(256, 9)
        )
        
    def forward(self, x, batch):
        x = self.input_mlp(x)
        x = global_max_pool(x, batch)
        x = self.output_mlp(x)
        x = x.view(-1, 3, 3)
        id_matrix = torch.eye(3).to(x.device).view(1, 3, 3).repeat(x.shape[0], 1, 1)
        x = id_matrix + x
        return x

In [10]:
class FeatureTNet(nn.Module):
    def __init__(self):
        super(FeatureTNet, self).__init__()
        self.input_mlp = nn.Sequential(
            nn.Linear(64, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, 1024), nn.BatchNorm1d(1024), nn.ReLU(),
        )
        self.output_mlp = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(),
            nn.Linear(256, 64*64)
        )
        
    def forward(self, x, batch):
        x = self.input_mlp(x)
        x = global_max_pool(x, batch)
        x = self.output_mlp(x)
        x = x.view(-1, 64, 64)
        id_matrix = torch.eye(64).to(x.device).view(1, 64, 64).repeat(x.shape[0], 1, 1)
        x = id_matrix + x
        return x

In [11]:
class PointNetClassification(nn.Module):
    def __init__(self):
        super(PointNetClassification, self).__init__()
        self.input_tnet = InputTNet()
        self.mlp1 = nn.Sequential(
            nn.Linear(3, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 64), nn.BatchNorm1d(64), nn.ReLU(),
        )
        self.feature_tnet = FeatureTNet()
        self.mlp2 = nn.Sequential(
            nn.Linear(64, 64), nn.BatchNorm1d(64), nn.ReLU(),
            nn.Linear(64, 128), nn.BatchNorm1d(128), nn.ReLU(),
            nn.Linear(128, 1024), nn.BatchNorm1d(1024), nn.ReLU(),
        )
        self.mlp3 = nn.Sequential(
            nn.Linear(1024, 512), nn.BatchNorm1d(512), nn.ReLU(), nn.Dropout(p=0.3),
            nn.Linear(512, 256), nn.BatchNorm1d(256), nn.ReLU(), nn.Dropout(p=0.3),
            nn.Linear(256, 20)
        )
        
    def forward(self, batch_data,batch_label):
        x = batch_data
        input_transform = self.input_tnet(x, batch_label)
        transform = input_transform[batch_label, :, :]
        x = torch.bmm(transform, x.view(-1, 3, 1)).view(-1, 3)
        
        x = self.mlp1(x)
        
        feature_transform = self.feature_tnet(x, batch_label)
        transform = feature_transform[batch_label, :, :]
        x = torch.bmm(transform, x.view(-1, 64, 1)).view(-1, 64)

        x = self.mlp2(x)        
        x = global_max_pool(x,batch_label)
        x = self.mlp3(x)
        
        return x, input_transform, feature_transform

In [13]:
import torch
from torch.utils.tensorboard import SummaryWriter
from torch_geometric.data import DataLoader as DataLoader

num_epoch = 400
batch_size = 32

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
model = PointNetClassification()
model = model.to(device)

optimizer = torch.optim.Adam(lr=1e-4, params=model.parameters())
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=num_epoch // 4, gamma=0.5)

log_dir = "logs/log_MSR_classification"
#log_dir.mkdir(exist_ok=True)
writer = SummaryWriter(log_dir=log_dir)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

criteria = torch.nn.CrossEntropyLoss()

In [14]:
from tqdm import tqdm

for epoch in tqdm(range(num_epoch)):
    model = model.train()
    
    losses = []
    for batch_data, y in tqdm(train_dataloader, total=len(train_dataloader)):
        this_batch_size =  len(y)

        batch_label = torch.zeros([this_batch_size*1024])
        for i in range(this_batch_size):
            batch_label[1024 * i : 1024 * (i + 1)] = i

        batch_data = batch_data.reshape((1024 * this_batch_size,3)).to(device)
        batch_label = batch_label.type(torch.LongTensor).to(device)
        pred_y, _, feature_transform = model(batch_data, batch_label)
        true_y = y.detach().to(device)

        class_loss = criteria(pred_y, true_y)
        accuracy = float((pred_y.argmax(dim=1) == true_y).sum()) / float(this_batch_size)

        id_matrix = torch.eye(feature_transform.shape[1]).to(feature_transform.device).view(1, 64, 64).repeat(feature_transform.shape[0], 1, 1)
        transform_norm = torch.norm(torch.bmm(feature_transform, feature_transform.transpose(1, 2)) - id_matrix, dim=(1, 2))
        reg_loss = transform_norm.mean()

        loss = class_loss + reg_loss * 0.001
        
        losses.append({
            "loss": loss.item(),
            "class_loss": class_loss.item(),
            "reg_loss": reg_loss.item(),
            "accuracy": accuracy,
            "seen": float(this_batch_size)})
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
    if (epoch % 50 == 0):
        model_path = log_dir + f"model_{epoch:06}.pth"
        torch.save(model.state_dict(), model_path)
    
    loss = 0
    class_loss = 0
    reg_loss = 0
    accuracy = 0
    seen = 0
    for d in losses:
        seen = seen + d["seen"]
        loss = loss + d["loss"] * d["seen"]
        class_loss = class_loss + d["class_loss"] * d["seen"]
        reg_loss = reg_loss + d["reg_loss"] * d["seen"]
        accuracy = accuracy + d["accuracy"] * d["seen"]
    loss = loss / seen
    class_loss = class_loss / seen
    reg_loss = reg_loss / seen
    accuracy = accuracy / seen
    taccuracy = accuracy
    writer.add_scalar("train_epoch/loss", loss, epoch)
    writer.add_scalar("train_epoch/class_loss", class_loss, epoch)
    writer.add_scalar("train_epoch/reg_loss", reg_loss, epoch)
    writer.add_scalar("train_epoch/accuracy", accuracy, epoch)
    with torch.no_grad():
        model = model.eval()

        losses = []
        for batch_data,y in tqdm(test_dataloader, total=len(test_dataloader)):
            this_batch_size =  len(y)

            batch_label = torch.zeros([this_batch_size*1024])
            for i in range(this_batch_size):
                batch_label[1024 * i : 1024 * (i + 1)] = i

            batch_data = batch_data.reshape((1024 * this_batch_size,3)).to(device)
            batch_label = batch_label.type(torch.LongTensor).to(device)


            pred_y, _, feature_transform = model(batch_data,batch_label)
            true_y = y.detach().to(device)

            class_loss = criteria(pred_y, true_y)
            accuracy =float((pred_y.argmax(dim=1) == true_y).sum()) / float(this_batch_size)

            id_matrix = torch.eye(feature_transform.shape[1]).to(feature_transform.device).view(1, 64, 64).repeat(feature_transform.shape[0], 1, 1)
            transform_norm = torch.norm(torch.bmm(feature_transform, feature_transform.transpose(1, 2)) - id_matrix, dim=(1, 2))
            reg_loss = transform_norm.mean()

            loss = class_loss + reg_loss * 0.001

            losses.append({
                "loss": loss.item(),
                "class_loss": class_loss.item(),
                "reg_loss": reg_loss.item(),
                "accuracy": accuracy,
                "seen": float(this_batch_size)})
            
        loss = 0
        class_loss = 0
        reg_loss = 0
        accuracy = 0
        seen = 0
        for d in losses:
            seen = seen + d["seen"]
            loss = loss + d["loss"] * d["seen"]
            class_loss = class_loss + d["class_loss"] * d["seen"]
            reg_loss = reg_loss + d["reg_loss"] * d["seen"]
            accuracy = accuracy + d["accuracy"] * d["seen"]
        loss = loss / seen
        class_loss = class_loss / seen
        reg_loss = reg_loss / seen
        accuracy = accuracy / seen
        writer.add_scalar("test_epoch/loss", loss, epoch)
        writer.add_scalar("test_epoch/class_loss", class_loss, epoch)
        writer.add_scalar("test_epoch/reg_loss", reg_loss, epoch)
        writer.add_scalar("test_epoch/accuracy", accuracy, epoch)

        print("epoch {}: train{}  val{}".format(epoch,taccuracy,accuracy))

  0%|          | 1/400 [00:11<1:18:00, 11.73s/it]

epoch 0: train0.04054054054054054  val0.0


  0%|          | 2/400 [00:16<50:05,  7.55s/it]  

epoch 1: train0.08108108108108109  val0.09947643979057591


  1%|          | 3/400 [00:21<41:41,  6.30s/it]

...

epoch 397: train0.918918918918919  val0.7801047120418848


100%|█████████▉| 399/400 [23:48<00:04,  4.15s/it]

epoch 398: train0.8851351351351351  val0.774869109947644


100%|██████████| 400/400 [23:52<00:00,  3.58s/it]

epoch 399: train0.8986486486486487  val0.7774869109947644


## TODO

- 固定長に変換
- モデルを少し変える
- 学習